In [ ]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
#from tensorflow.keras import Model, layers
from tensorflow.keras.layers import Dense, Conv1D, LSTM, Input, TimeDistributed, Conv2D
from tensorflow.keras.models import Model
import numpy as np
import random

num_classes = 6 # 5 sequence class or not.
seq_max_len = 140 # Maximum sequence length.
seq_min_len = 100 # Minimum sequence length (before padding).
masking_val = -1 # -1 will represents the mask and be used to pad sequences to a common max length.
max_value = 10000 # Maximum int value.??

# Training Parameters
learning_rate = 0.001
training_steps = 2000
batch_size = 64
display_step = 100

# Network Parameters
num_units = 32 # number of neurons for the LSTM layer.(output dimension)


# data generator from original image
def sequence_data():
    pass


# dataset
train_data = tf.data.Dataset.from_generator(toy_sequence_data, output_types=(tf.float32, tf.float32))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)


# LSTM model
class LSTM(Model):
    def __init__(self):
        super(LSTM, self).__init__()
        self.lstm = LSTM(unit=num_units) # dimension of output space
        self.out = Dense(num_classes) # output fully connected layer

    # forward pass
    def call(self, x, is_training=False):
        x = tf.reshape(x, shape=[-1, seq_max_len, 1]) # (batch size, sequence length, num features)

        if not is_training:
            x = tf.nn.softmax(x)

        return x

# build LSTM model
lstm_net = LSTM()

# cross entroy loss
def cross_entropy_loss(x,y):
    y = tf.cast(y, tf.int64)
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=x)
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)

# Adam optimizer.
optimizer = tf.optimizers.Adam(learning_rate)


def run_optimization(x, y):
    # Wrap computation inside a GradientTape for automatic differentiation.
    with tf.GradientTape() as g:
        # Forward pass.
        pred = lstm_net(x, is_training=True)
        # Compute loss.
        loss = cross_entropy_loss(pred, y)
        
    # Variables to update, i.e. trainable variables.
    trainable_variables = lstm_net.trainable_variables

    # Compute gradients.
    gradients = g.gradient(loss, trainable_variables)
    
    # Update weights following gradients.
    optimizer.apply_gradients(zip(gradients, trainable_variables))

# number of step 만큼 학습 
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    # Run the optimization to update W and b values.
    run_optimization(batch_x, batch_y)
    
    if step % display_step == 0 or step == 1:
        pred = lstm_net(batch_x, is_training=True)
        loss = cross_entropy_loss(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

